## Clustering draft 1

**General idea** : test the functions defined in the module1.py file and make a first application.

**General plan** : 
1. Import the data using the module YahooFinance 

2. Repeat the clustering several times to get several clustering outcomes

3. Compute the weights of each stocks within each cluster 

4. Compute the returns of each portfolio, where one portfolio corresponds to one clustering and is composed of 5 big assets (which are the 5 clusters) **in this notebook, we first fix the number of clusters to be equal to 5**

Here are the main package we use in this notebook

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from bs4 import BeautifulSoup
import requests 
from pypfopt.efficient_frontier import EfficientFrontier


We also import the module1.py file to use the function we defined there.

In [2]:
pip install git+https://github.com/robertmartin8/PyPortfolioOpt.git

  Cloning https://github.com/robertmartin8/PyPortfolioOpt.git to /private/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/pip-req-build-eicjrdld
  Running command git clone --filter=blob:none --quiet https://github.com/robertmartin8/PyPortfolioOpt.git /private/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/pip-req-build-eicjrdld
  Resolved https://github.com/robertmartin8/PyPortfolioOpt.git to commit 30ab57147ba61eddc8301294a5b1c5ef260b23fa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [3]:
## we make some manipulations to correctly import module1

import os
import sys

# on a obtenu le chemin absolu vers notre répertoire en utilisant
# le terminal (ls, pwd, cd)
module_1_directory = '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/code'

# Ajouter le chemin du répertoire 'code' au chemin de recherche Python
sys.path.append(module_1_directory)

# Maintenant, vous pouvez importer module1
import module1

### 1. Importation of data

We download various types of data (Open, High, Low, Close, Adj Close, Volume) for **496** assets between two periods (start, end) into a pandas dataframe denoted as df.

We then store the returns of each of these assets in the 'data'.

#### 1.1. Scraping the names of the assets of interest

- This is important because to import financial data for assets on Yahoo Finance, you need the tickers' names. This scraping work was performed in the file *get_S&P500_symbols.ipynb* and generated the *S&P500_symbols.csv* file.

- We rely on the csv package to read the *S&P500_symbols.csv* file and to create a list containing all the tickers

In [4]:
import csv 

with open('S&P500_symbols.csv', mode='r') as file:
    # Create a CSV reader
    csv_reader = csv.reader(file)
    
    # Transform the CSV data into a list
    stock_symbols = []
    for row in csv_reader:
        stock_symbols.append(row[0])

# Close the file
file.close()

stock_symbols.pop(0)

'Symbols of S&P 500'

In [5]:
stock_symbols

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADM',
 'ADBE',
 'ADP',
 'AES',
 'AFL',
 'A',
 'ABNB',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'WRB',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BG',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CDAY',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',


#### 1.2. Creating the dataframe using pandas and yfinance 

In this case, we focus on the financial data of asset during the year 2022

In [6]:
n_stocks = len(stock_symbols) # number of stocks = 502

start = "2022-01-01" # start date
end = "2022-12-01" # end date

df = pd.DataFrame(yf.download(stock_symbols, start, end)) # data on the 198 assets
data = np.log(df["Close"]/df["Open"]).transpose() # compute the returns of these assets
data = data.dropna()



[*********************100%%**********************]  502 of 502 completed


5 Failed downloads:
['KVUE', 'VLTO', 'GEHC']: Exception("%ticker%: Data doesn't exist for startDate = 1641013200, endDate = 1669870800")
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2022-01-01 -> 2022-12-01)')
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


In [7]:
data

Date,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-10,2022-01-11,2022-01-12,2022-01-13,2022-01-14,...,2022-11-16,2022-11-17,2022-11-18,2022-11-21,2022-11-22,2022-11-23,2022-11-25,2022-11-28,2022-11-29,2022-11-30
A,-0.015976,-0.028044,-0.014895,0.001812,-0.026984,0.012966,0.011247,0.011503,-0.029123,0.004433,...,-0.007247,-0.006143,-0.003346,-0.003302,0.031605,-0.008461,0.011985,-0.023170,-0.006208,0.023503
AAL,0.028125,-0.010460,-0.023804,-0.017085,0.027875,-0.023667,0.007388,-0.035054,0.032049,-0.032982,...,-0.025229,0.009363,-0.014836,-0.012200,0.009342,0.030988,0.006920,-0.036213,0.017839,0.021717
AAPL,0.023234,-0.016173,-0.026459,-0.004061,-0.004173,0.018227,0.015890,-0.003356,-0.020635,0.010046,...,-0.002283,0.028876,-0.006719,-0.014422,0.013744,0.010781,-0.001349,-0.006359,-0.021860,0.045822
ABBV,0.000074,-0.001257,0.006424,-0.008615,-0.002739,0.009651,0.008211,0.005624,-0.011023,0.018721,...,-0.010285,0.007851,0.008489,0.010944,0.013229,-0.003507,0.000188,-0.009861,-0.000695,0.014939
ABNB,0.043696,-0.019447,-0.045972,-0.003375,0.042253,0.005929,0.047987,-0.004120,-0.025180,-0.010675,...,-0.030833,-0.025159,-0.030917,-0.013491,0.018538,0.011135,0.007193,-0.017681,-0.007520,0.063456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,-0.013459,0.000583,-0.014505,0.006742,-0.007434,-0.009694,-0.018652,-0.015062,-0.003719,-0.008308,...,0.004771,-0.000651,0.003054,-0.005789,0.003930,0.002624,0.005309,0.001345,-0.000317,0.018276
ZBH,0.013880,0.011876,-0.008349,-0.015088,0.007441,-0.003461,0.006329,-0.019556,0.014868,0.004169,...,-0.006097,-0.005603,0.004216,0.009739,0.007048,0.013995,0.006396,-0.003986,0.015583,0.005176
ZBRA,-0.013912,0.004862,-0.042476,0.000288,-0.039303,0.015130,0.006370,-0.002837,-0.026315,0.014806,...,-0.005133,0.013099,-0.012765,0.011939,0.014081,0.029699,0.005162,0.002307,-0.011070,0.013634
ZION,0.005620,-0.000605,-0.003768,0.027053,0.013068,-0.015973,0.001425,-0.004547,-0.005551,0.014112,...,-0.022113,-0.004521,-0.021279,0.000985,0.001562,0.006817,0.004066,-0.022052,0.012026,0.020077


Due to missing values in the data, 5 stocks have been excluded from the intial 502 we considered. Thus the shape of the dataframe is (496, 230) 






### 2. We repeat the clustering step several times to start preparing multiple micro-portfolios

**General idea**: The multiple_clustering function takes as arguments:

1. *n_repeat*: an integer that corresponds to the number of times we want to train (.fit()) the model passed as an argument on the data.

2. *data*: a pandas dataframe (like the one we generated above) that corresponds to the financial data on which the model will be trained.

3. *model*: a sklearn clustering model (for now, we will test it with KMeans).

4. *model_name*: the name of the model, necessary for creating the pipeline properly.

**Outcome**: a pandas dataframe storing the results of the multiple clustering and a dictionary containing the centroids of the clusters each time 

We first make the test using the K-Means model and fix we fix the number of clusters to 5

In [8]:
model = KMeans(n_clusters=5)
model_name = 'kmeans'

Y, C = module1.multiple_clusterings(10, data, model, model_name)
print(Y, C)

/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

      Clustering n°1  Clustering n°2  Clustering n°3  Clustering n°4  \
A                  1               4               1               1   
AAL                0               3               4               3   
AAPL               1               4               1               2   
ABBV               2               1               0               4   
ABNB               0               0               3               3   
...              ...             ...             ...             ...   
YUM                1               4               1               4   
ZBH                1               4               1               1   
ZBRA               1               0               3               2   
ZION               3               3               1               1   
ZTS                1               4               1               4   

      Clustering n°5  Clustering n°6  Clustering n°7  Clustering n°8  \
A                  3               0               1           

/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [9]:
Y

,Clustering n°1,Clustering n°2,Clustering n°3,Clustering n°4,Clustering n°5,Clustering n°6,Clustering n°7,Clustering n°8,Clustering n°9,Clustering n°10
A,1,4,1,1,3,0,1,3,3,1
AAL,0,3,4,3,4,4,2,4,0,0
AAPL,1,4,1,2,1,0,4,3,3,1
ABBV,2,1,0,4,2,3,0,0,1,2
ABNB,0,0,3,3,4,2,2,4,0,0
...,...,...,...,...,...,...,...,...,...,...
YUM,1,4,1,4,2,0,1,3,3,1
ZBH,1,4,1,1,3,0,1,3,3,1
ZBRA,1,0,3,2,1,2,4,2,4,4
ZION,3,3,1,1,3,0,1,3,3,1


### 3. Compute the weights of the stocks in each cluster 

#### 3.1. Get the cluster compositions each time 

**Why doing so?**: the purpose of this work is to compute the returns of each cluster (seen as a new "fictive" asset composed of "real" assets) and this latter task is easier if we know the cluster composition in terms of tickers (because in the financial database we have data given a ticker).

In [10]:
Y_symbol = module1.cluster_composition(Y)
Y_symbol

,Clustering n°1,Clustering n°2,Clustering n°3,Clustering n°4,Clustering n°5,Clustering n°6,Clustering n°7,Clustering n°8,Clustering n°9,Clustering n°10
Cluster 1,"[A, AAPL, ABT, ACN, ADBE, ADI, ADP, AES, AKAM,...","[A, AAPL, ABT, ACN, ADP, AES, AJG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AJG, AKAM, ...","[A, ACN, AES, AIG, AKAM, ALLE, AMCR, AME, AMP,...","[A, ACN, AIG, ALLE, AME, AMP, AOS, APD, APH, A...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AJG, AKAM, ...","[A, ABT, ACN, ADP, AES, AIG, AKAM, ALLE, AMCR,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,..."
Cluster 2,"[AAL, ABNB, ADSK, ALB, ALGN, AMAT, AMD, AMZN, ...","[AAL, AIG, ALK, AMP, APTV, AXP, BA, BAC, BBWI,...","[AAL, ALK, APTV, BA, BBWI, BBY, BKNG, BWA, BX,...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ALK, BA, BBWI, BKNG, CCL, CZR, DAL...","[AAL, ALK, APTV, BA, BBWI, BBY, BKNG, BWA, BX,...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ..."
Cluster 3,"[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, AJG, ALL...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, ALL, AMC...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, ALL, AMG...","[AAPL, ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, ...","[AAPL, ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, ...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, ALL, AMG...","[AAPL, ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, ...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, AJG, ALL...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, AJG, ALL...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, AJG, ALL..."
Cluster 4,"[AIG, ALK, AMP, AVY, AXP, BA, BAC, BBY, BEN, B...","[ABNB, ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, ...","[ABNB, ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, ...","[ABBV, ABT, ACGL, ADM, ADP, AEE, AEP, AFL, AIZ...","[ABBV, ABT, ACGL, ADM, ADP, AEE, AEP, AES, AFL...","[ABNB, ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, ...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, AJG, ALL...","[ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, AMZN, ...","[ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, AMZN, ...","[ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, AMZN, ..."
Cluster 5,"[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ..."


In [11]:
Y_symbol.columns

Index(['Clustering n°1', 'Clustering n°2', 'Clustering n°3', 'Clustering n°4',
       'Clustering n°5', 'Clustering n°6', 'Clustering n°7', 'Clustering n°8',
       'Clustering n°9', 'Clustering n°10'],
      dtype='object')

#### 3.2. Compute the weights of each assets within a cluster

**Why doing so?**: because each stock in a cluster has a different weight. Stock are "representative" of the cluster in which they live to a different extent. So far, we built the weight of a stock based on its $\mathcal{L}^2$ distance from the cluster's centroid. We can therefore see the weights as a kind of "inertia contribution" of the stock in the cluster.

##### 3.2.1. Analysis of the centroids 

- The .multiple_clusterings() function returns two data frame, the first of which is a dataframe containing the centroid of each cluster. This centroid is an array containing the "average" return each day, where each daily average is taken on the returns of all the stocks contained in the corresponding cluster. 

- Here is an example:


In [12]:
C

,Clustering n°1,Clustering n°2,Clustering n°3,Clustering n°4,Clustering n°5,Clustering n°6,Clustering n°7,Clustering n°8,Clustering n°9,Clustering n°10
Cluster 1,"[0.22615354424308662, -1.0781069236142629, -1....","[-0.04314403662613201, -1.141064652478645, -1....","[-0.01860364388568653, -0.01862831818072793, 0...","[1.5002065476511288, 1.0570143644422916, -0.31...","[1.5002065476511288, 1.0570143644422916, -0.31...","[-0.25194842219191527, 0.12478920078893815, 0....","[-0.008305716486801529, -0.009357343405343894,...","[-0.01922149164208389, 0.005907972939479033, 0...","[1.0818821358026836, -0.10293663224269473, -0....","[1.1273320614067666, -0.2772005302773248, -0.8..."
Cluster 2,"[-0.6120939407232203, -0.4022968178626149, -0....","[0.025383919877601767, 0.06217753779094265, 0....","[-0.2402053300427742, 0.14258006702764833, 0.0...","[-0.08839945176544643, 0.4091547956640719, 0.1...","[-0.27430873857986804, -1.1521589358985447, -1...","[1.5002065476511288, 1.0570143644422916, -0.31...","[-0.21136983848476185, 0.2207378012174472, 0.0...","[1.5002065476511288, 1.0570143644422916, -0.31...","[-0.06837829265966026, -0.038266720460169476, ...","[-0.16104954283093928, 0.2459035963095379, 0.0..."
Cluster 3,"[-0.0249862522487604, 0.03766303221271663, 0.6...","[1.5002065476511288, 1.0570143644422916, -0.31...","[1.5293699302639876, 1.106898060194132, -0.357...","[-0.34293959129867296, -1.081741196678282, -1....","[-0.15644191990595732, -0.08941032285136294, 0...","[-0.029047816201335824, -1.0978525690757541, -...","[1.1244838244132374, -0.2749346042866737, -0.8...","[-0.17409859643615738, -1.138527299010478, -1....","[1.5002065476511288, 1.0570143644422916, -0.31...","[-0.074090118998784, -0.03892263003886425, 0.6..."
Cluster 4,"[0.37721568621731183, 0.769039885777627, 0.224...","[0.5224035761283713, 0.668274800623168, 0.0312...","[-0.029047816201335824, -1.0978525690757541, -...","[1.0828727511578433, -0.09841294961698888, -0....","[-0.06042157466449491, 0.43187827930980177, 0....","[-0.0042706263815413936, 0.008306660270957772,...","[1.5293699302639876, 1.106898060194132, -0.357...","[-0.22232290767337284, 0.16625200781813876, 0....","[-0.172799908965004, 0.23482159858664514, 0.03...","[1.5293699302639876, 1.106898060194132, -0.357..."
Cluster 5,"[1.5002065476511288, 1.0570143644422916, -0.31...","[-0.5790405586936084, -0.2601908028995867, -0....","[0.48980533486923755, 0.05938042252052507, -0....","[-0.12622382780150948, -0.11511783058636087, 0...","[1.2942035480892973, -0.5863746082213125, -0.9...","[0.4810519825776648, 0.0637070425456074, -0.59...","[-0.1983973612611435, -1.118144619802161, -1.1...","[1.0818821358026836, -0.10293663224269473, -0....","[-0.25594876226768776, -1.1625746749393877, -1...","[-0.23752576235501585, -1.158431753465747, -1...."


##### 3.2.2. Computing the weights of each cluster

- In module1, there are two functions *cluster_weights* and *gaussian_weights* which, given one cluster and its centroids, return the weights of the stocks in the cluster. These weights corresponds to a sort of distance of the stocks to the cluster's centroid.

- If we consider the previous example, we can consider the first cluster (Cluster 1 of the Clusering n°1) and compute the weights of the stocks in this cluster. 


In [13]:
Y_symbol.head(2)

,Clustering n°1,Clustering n°2,Clustering n°3,Clustering n°4,Clustering n°5,Clustering n°6,Clustering n°7,Clustering n°8,Clustering n°9,Clustering n°10
Cluster 1,"[A, AAPL, ABT, ACN, ADBE, ADI, ADP, AES, AKAM,...","[A, AAPL, ABT, ACN, ADP, AES, AJG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AJG, AKAM, ...","[A, ACN, AES, AIG, AKAM, ALLE, AMCR, AME, AMP,...","[A, ACN, AIG, ALLE, AME, AMP, AOS, APD, APH, A...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AJG, AKAM, ...","[A, ABT, ACN, ADP, AES, AIG, AKAM, ALLE, AMCR,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,..."
Cluster 2,"[AAL, ABNB, ADSK, ALB, ALGN, AMAT, AMD, AMZN, ...","[AAL, AIG, ALK, AMP, APTV, AXP, BA, BAC, BBWI,...","[AAL, ALK, APTV, BA, BBWI, BBY, BKNG, BWA, BX,...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ALK, BA, BBWI, BKNG, CCL, CZR, DAL...","[AAL, ALK, APTV, BA, BBWI, BBY, BKNG, BWA, BX,...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ..."


#### **Example**: 
- To take a basic example, we consider the first cluster of the first clustering (if we refer to the Y_symbol cell above this corresponds to *Y_symbol.iloc[0, 0]* for the tickers and to *C.iloc[0, 0]* for the centroids).

- We then compute the $\mathcal{L}^2$ weights and the Gaussian weights for this cluster 



In [14]:
    ## We consider the first cluster of the first clustering 
    #  (location [0, 0])

cluster = Y_symbol.iloc[0,0]
centroid = C.iloc[0,0]

    ## We then compute the weights (L2 and Gaussian) corresponding to 
    #  the stocks in this cluster 

weights_L2 = module1.cluster_weights(cluster, centroid, data)
weights_gaussian = module1.gaussian_weights(cluster, centroid, data)


In [16]:
weights_gaussian

,0,1,2,3,4,5,6,7,8,9,...,137,138,139,140,141,142,143,144,145,146
0,0.001328,0.006218,0.000073,0.001568,0.154571,0.019138,0.000103,0.000385,0.00027,0.000273,...,0.000985,0.000228,0.001145,0.000117,0.000127,0.000439,0.00009,0.000219,0.041198,0.000178


In [17]:
cluster_composition = cluster 
micro_portfolio_return = pd.DataFrame(index=cluster_composition, columns=data.columns).transpose()
micro_portfolio_return

,A,AAPL,ABT,ACN,ADBE,ADI,ADP,AES,AKAM,ALLE,...,VRSN,WAT,WST,WTW,XRAY,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- We make sure that the weights add up to one

In [18]:
print(weights_L2.sum(axis=1))
print(weights_gaussian.sum(axis=1))

0    1.0
dtype: float64
0    1.0
dtype: float64


In [19]:
weights_gaussian

,0,1,2,3,4,5,6,7,8,9,...,137,138,139,140,141,142,143,144,145,146
0,0.001328,0.006218,0.000073,0.001568,0.154571,0.019138,0.000103,0.000385,0.00027,0.000273,...,0.000985,0.000228,0.001145,0.000117,0.000127,0.000439,0.00009,0.000219,0.041198,0.000178


In [37]:
weights_gaussian.applymap(lambda x: x if x > 0.003 else None)

/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_47509/803272374.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weights_gaussian.applymap(lambda x: x if x > 0.003 else None)


,0,1,2,3,4,5,6,7,8,9,...,226,227,228,229,230,231,232,233,234,235
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,0.003765,None


### 4. Compute the returns of each portfolio, 

- **Idead**: We considered that one portfolio corresponds to one clustering and is composed of 5 big assets (which are the 5 clusters) [*in this notebook, we first fix the number of clusters to be equal to 5*]

To have an idea of how we built the clustering_return() function, see the shape of cluter_data (which corresponds to the return data associated with the tickers in the cluster) and the weights of the stocks in this cluster 

In [17]:
cluster_data = data.loc[cluster]
print(cluster_data.shape)
print(weights_gaussian.shape)

(236, 230)
(1, 236)


#### **Remarks**:

- We see that the to get the return of each clusters, we have to multiply each columns of the cluster_data dataframe by the weights of the stocks (take the transpose of the line dataframe).

- The **clustering_return** function applies to a **clustering** i.e. one column of the Y_symbol dataframe (result of the cluster_composition() funciton). 

In [20]:
## We take as an example the first clustering of the Y_symbol dataframe
clustering_composition_1 = pd.DataFrame(Y_symbol.iloc[:, 0])
clustering_centroids_1 = pd.DataFrame(C.iloc[:, 0])

cluster_1 = clustering_composition_1.iloc[0, 0]
centroid_1 = clustering_centroids_1.iloc[0, 0]

In [21]:
module1.clustering_return(clustering_composition_1, clustering_centroids_1, data)

,Clustering n°1
Cluster 1,"[-0.013857069958154765, -0.015464240824801814,..."
Cluster 2,"[-0.008972811717595295, -0.021533893130766396,..."
Cluster 3,"[-0.0019549793752205572, 0.0034057515998063915..."
Cluster 4,"[0.009153693046501994, 0.01858037910765008, -0..."
Cluster 5,"[0.037842947899023135, 0.0335931972565851, -0...."


In [22]:
Z = module1.clustering_return(Y_symbol, C, data)
Z

,Clustering n°1,Clustering n°2,Clustering n°3,Clustering n°4,Clustering n°5,Clustering n°6,Clustering n°7,Clustering n°8,Clustering n°9,Clustering n°10
Cluster 1,"[-0.013857069958154765, -0.015464240824801814,...","[-0.019622484233164636, -0.015749074769608386,...","[-0.007348386642801764, 0.0055327369323147635,...","[0.0023081930061286827, 0.023466881248934708, ...","[0.002545375819969189, 0.023720594925962023, -...","[-0.007795018155345543, 0.004981767530357814, ...","[-0.0073928114629485805, 0.005442229701844507,...","[-0.007378490468619931, 0.005024468823267883, ...","[0.0032374027106475585, 0.011053019086576429, ...","[0.004286181156365292, 0.013216583927586614, -..."
Cluster 2,"[-0.008972811717595295, -0.021533893130766396,...","[0.007590688011789732, 0.02012687354200748, -0...","[0.0036992088433154633, 0.0059424651730596596,...","[0.016739396421055992, 0.004954192559407782, -...","[0.008858926891910684, -0.04642123234350873, -...","[0.0015363914135737245, 0.008352095912835821, ...","[0.017361991975001437, 0.00029036312693824905,...","[0.015297251094955482, 0.006430439735358495, -...","[0.016107645169329395, 0.004656130839774155, -...","[0.017578391562188777, 0.00013188275977814908,..."
Cluster 3,"[-0.0019549793752205572, 0.0034057515998063915...","[0.00063773433888515, 0.0034711513355258905, -...","[0.010297753776205143, 0.0011053086764828963, ...","[-0.01998666554655074, -0.03125646969560483, -...","[-0.012333992781297968, -0.0189943718847869, -...","[-0.004119351503034082, -0.0020710989305465152...","[-0.004374362268854975, -0.028126661724027044,...","[-0.005563497238729689, -0.003068181025504284,...","[0.0004989282793566697, 0.0031979456939057295,...","[-0.002362333659318029, 0.0029333039932069567,..."
Cluster 4,"[0.009153693046501994, 0.01858037910765008, -0...","[0.0049156312780921935, -0.02863096890763489, ...","[-0.010218199917159755, -0.0338736552120599, -...","[-0.0015852279061343317, -0.000313595245266273...","[-0.004742261766836646, 0.002249008399781223, ...","[-0.0008053635545129616, -0.015202794323552801...","[0.010303767088387924, 0.0011197797038407895, ...","[-0.005590634344493105, -0.016908531725988347,...","[-0.006462803356326924, -0.016936257344114587,...","[-7.636578502436815e-05, 0.00967567030383951, ..."
Cluster 5,"[0.037842947899023135, 0.0335931972565851, -0....","[0.02579382289377349, 0.019372463271760603, -0...","[0.027658294911561142, 0.02134589816559207, -0...","[0.024520993004813887, 0.0167845022695817, -0....","[0.023888356363929143, 0.011019076020662167, -...","[0.02767688581858046, 0.021224148290716928, -0...","[0.027107823624382087, 0.0214703944146061, -0....","[0.026465146271765005, 0.0173023145097758, -0....","[0.027107594033754925, 0.02136454254908335, -0...","[0.02731903417658724, 0.021599662445275315, -0..."


In [23]:
len(Z.iloc[0,0])

230

In [40]:
np_returns = np.array([returns.iloc[i] for i in range(len(returns.values))])

In [41]:
np_returns.T

array([[ 0.00246421,  0.01754678,  0.01548605, -0.00700052,  0.02758107],
       [ 0.01471503, -0.00037659,  0.00227068, -0.01755431,  0.02206587],
       [-0.01891195, -0.03363215, -0.00839963, -0.03562105, -0.03470268],
       ...,
       [-0.01014273, -0.01391672,  0.00704841, -0.00180774, -0.00029462],
       [ 0.00643239,  0.01121268,  0.01403385, -0.00831259,  0.00617351],
       [ 0.016823  ,  0.02662736,  0.00420253,  0.04936655, -0.00670652]])

In [42]:
expected_returns = np.mean(np_returns.T, axis=0)
cov_matrix = np.cov(np_returns.T, rowvar=False)

In [43]:
expected_returns 

array([-0.00010188, -0.0005411 ,  0.00033631, -0.00014543,  0.00154747])

In [45]:
module1.markowitz(expected_returns, cov_matrix)


ValueError: at least one of the assets must have an expected return exceeding the risk-free rate